import

In [1]:
import os

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate

xionic-ko-llama

In [2]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [3]:
response = client.chat.completions.create(
    model="xionic-ko-llama-3-70b",
    messages=[
        {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
        {"role": "user", "content": "아이브 리더가 활동했던 이전 걸그룹의 명칭은?"}
    ]
)

print (response)

ChatCompletion(id='cmpl-b68b656c7fa0488f933e78d9c9bf89b9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="아이브 리더인 안유진은 데뷔 전 걸그룹 IZ*ONE에서 활동했습니다. IZ*ONE은 2018년 Mnet에서 방영된 리얼리티 프로그램 '프로듀스 48'을 통해 결성된 프로젝트 걸그룹으로, 2018년 10월 29일 데뷔 했습니다. 이 그룹은 프로듀스 48에 참가한 96명의 연습생 중 최종 12명을 선발하여 구성됐으며, 안유진은 이 중 한 명으로 선발됐습니다. IZ*ONE은 2021년 4월 29일까지 활동했으며, 이후 안유진은 아이브의 리더로 데뷔하게 됩니다.", role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1721259565, model='xionic-ko-llama-3-70b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=166, prompt_tokens=56, total_tokens=222))


문서 가져오기

In [4]:
PATH = "C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/data"
Product_data = os.path.join(
    PATH, "년도별 신제품 리스트_냉장-240425.xlsx"
)

In [7]:
def read_prompt_template(file_path: str) -> str:
    with open(file_path, "rb") as f:
        prompt_template = f.read()

    return prompt_template

In [6]:
# def create_chain(template_path):
#     prompt_template = read_prompt_template(template_path)
    
#     chain = LLMChain(
#         llm="xionic-ko-llama-3-70b",
#         prompt=ChatPromptTemplate.from_template(template=prompt_template),
#         max_tokens=150,
#         temperature=0.7,
#         stop=None
#     )
    
#     return chain

In [11]:
def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm = llm,
        prompt = ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,        
    )

In [10]:
llm = client

In [ ]:
product_data = create_chain(
    llm = llm
    
)